In [40]:
import pandas as pd
import sklearn as sk
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
%matplotlib inline


from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib

In [41]:
# Load raw data
train_feat = pd.read_csv('train_features.csv')
train_sal = pd.read_csv('train_salariesLGR.csv')
test_feat = pd.read_csv('test_features.csv')
train_sal.head()

,jobId,salary
0,JOB1362684407687,130
1,JOB1362684407688,101
2,JOB1362684407689,137
3,JOB1362684407690,142
4,JOB1362684407691,163


In [42]:
#train_sal['salarys'] = train_sal['salarys'] [train_sal['salary'] < 100 ] =0.1
#train_sal['salarys'] = train_sal['salarys'] [train_sal['salary'] <= 150 ] =0.4
#train_sal['salarys'] = train_sal['salarys'] [train_sal['salary'] <= 200 ] =0.9
#train_sal['salarys'] = train_sal['salarys'] [train_sal['salary'] > 200 ] =1

# train_sal[train_sal['salary'] > 200] =1
# train_sal[train_sal['salary'] = 150] =0.4
# train_sal[train_sal['salary'] <= 100] =0.1

train_sal['sal1'] = train_sal['salary'] <= 150
train_sal['sal1'] [train_sal['sal1'] == True]=0.4
train_sal['sal2'] = train_sal['salary'] < 100 
train_sal['sal2'] [train_sal['sal2'] == True]=0.1
train_sal['sal3'] = train_sal['salary'] > 150
train_sal['sal3'] [train_sal['sal3'] == True]=0.9
# train_sal['sal4'] = train_sal['salary'] > 200
# train_sal['sal4'] [train_sal['sal4'] == True]=1

a = train_sal['sal1'] + train_sal['sal2'] + train_sal['sal3'] # + train_sal['sal4']
# print(a)
train_sal['sals'] = a
train_sal.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,jobId,salary,sal1,sal2,sal3,sals
0,JOB1362684407687,130,0.4,0.0,0.0,0.4
1,JOB1362684407688,101,0.4,0.0,0.0,0.4
2,JOB1362684407689,137,0.4,0.0,0.0,0.4
3,JOB1362684407690,142,0.4,0.0,0.0,0.4
4,JOB1362684407691,163,0.0,0.0,0.9,0.9


In [43]:
train_data = pd.merge(train_feat, train_sal, on ='jobId')
train_data.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary,sal1,sal2,sal3,sals
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130,0.4,0.0,0.0,0.4
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101,0.4,0.0,0.0,0.4
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137,0.4,0.0,0.0,0.4
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142,0.4,0.0,0.0,0.4
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163,0.0,0.0,0.9,0.9


In [44]:
train_data = train_data.drop('jobId', axis = 1)
train_data = train_data.drop('companyId', axis = 1)
train_data = train_data.drop('sal1', axis = 1)
train_data = train_data.drop('sal2', axis = 1)
train_data = train_data.drop('sal3', axis = 1)
test_feat = test_feat.drop('jobId', axis = 1)
test_feat = test_feat.drop('companyId', axis = 1)

In [45]:
train_data = pd.get_dummies(train_data)
# <=100 = 0.1 
# <=150 = 0.4
# <=200 = 0.9
# >200 = 1

train_data.head()

,yearsExperience,milesFromMetropolis,salary,sals,jobType_CEO,jobType_CFO,jobType_CTO,jobType_JANITOR,jobType_JUNIOR,jobType_MANAGER,...,major_MATH,major_NONE,major_PHYSICS,industry_AUTO,industry_EDUCATION,industry_FINANCE,industry_HEALTH,industry_OIL,industry_SERVICE,industry_WEB
0,10,83,130,0.4,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,3,73,101,0.4,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,10,38,137,0.4,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,8,17,142,0.4,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,8,16,163,0.9,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [50]:
x = train_data.drop('sals', axis = 1)
    
y = train_data['sals']
print(y)

0         0.4
1         0.4
2         0.4
3         0.4
4         0.9
5         0.4
6         0.9
7         0.5
8         0.5
9         0.4
10        0.4
11        0.4
12        0.5
13        0.9
14        0.5
15        0.9
16        0.5
17        0.4
18        0.4
19        0.5
20        0.5
21        0.5
22        0.4
23        0.5
24        0.9
25        0.4
26        0.9
27        0.4
28        0.4
29        0.5
         ... 
999970    0.9
999971    0.4
999972    0.5
999973    0.4
999974    0.4
999975    0.5
999976    0.4
999977    0.4
999978    0.5
999979    0.9
999980    0.5
999981    0.4
999982    0.4
999983    0.4
999984    0.4
999985    0.4
999986    0.4
999987    0.4
999988    0.5
999989    0.5
999990    0.5
999991    0.4
999992    0.4
999993    0.4
999994    0.4
999995    0.5
999996    0.9
999997    0.5
999998    0.4
999999    0.5
Name: sals, Length: 1000000, dtype: float64


In [48]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

print("Số lượng mẫu đào tạo:",x_train.shape[0])
print("Số lượng mẫu test :", x_test.shape[0])

Số lượng mẫu đào tạo: 800000
Số lượng mẫu test : 200000


In [51]:
# tạo đối tượng Logistic Regressor

lr = LogisticRegression()
lr.fit(x_train, y_train)
y_hat = lr.predict(x_test)
print("5 mức lương đầu tiên là : ", y_hat[0:5])
print('The R-square is: ', lr.score(x,y))

ValueError: Unknown label type: 'continuous'